# Тестовое задание на позицию iGaming Product Analyst

**1. Составьте список KPI метрик необходимых для понимания качества интернет-казино. Ответ аргументируйте.**

* **DAU, MAU & Sticky factor.** Динамика данных показателей демонстрирует увеличение количества активных игроков по мере развития продукта, насколько они лояльны и играют ежедневно, а также влияние маркетинговых активностей на вовлечение пользователей в иргу. DAU также полезен для отслеживания технических проблем (резкое снижение может говорить о каком-то сбое).

* **Gross or new users DAU/MAU.** Количество привлечённых пользователей - хороший показатель для демонстрации рекламных кампаний на привлечение новых игроков. Лучше всего смотреть в сегментации по маркетинговым активностям или каналам привлечения.

* **ARPU (ARPUDAU, ARPUMAU).** Динамика средней выручки хорошо демонстрирует развитие доходности онлайн-казино и влияние маркетинговых активностей на вовлечение пользователей в иргу с точки зрения их вложений.

* **LTV, CAC & ROMI.** Данные метрики помогают оценить эффективность маркетинговых компаний и качество привлечённых пользователей с точки зрения дохода. Также LTV необходим для оценки ROI, т.е. окупаемости продукта в целом и прогнозирования трат и доходов на разработку и внедрение новых фич.

* **Lifetime, Retention Rate & Churn Rate.** Данные показатели связаны с удержанием игроков и демонстрируют их интерес, волечённость и лояльность с сервису. Также их резкое уменьшение/увеличение может показать реакцию игроков на новую фичу или технический сбой.

* **Conversion Rate.** Зависит от того, какую конверсию смотрят. Например, конверсия из неплатещего в платящего показывает высокий интерес к игре, мотивирующий на осуществление покупки. Конверсия в использование новой фичи демонстрирует насколько фича зашла пользователям. Конверсия из первого посещения в игру демонстрирует качество привлечённых пользователей. Конверсия из первой игры в, например, третью за сессию - демонстрирует уровень затягивания в игровой процесс.

* **GGR & NGR (gross & net gaming revenue).** Данные показатели важны и для понимания игрового опыта и для окупаемости казино. Снижение/увеличение данных показателей может говорить о качестве игр с точки зрения интереса, вовлечения и честности. Также это основные показатели для оценки, пронозировании и оптимизации доходов и расходов казино.

* **Average Deposit/Withdrawal Amount, Deposit/Withdrawal Frequency & Total Deposit/Withdrawal Volume.** Данные метрики непосредственно связаны с лояльность пользователей, их вовлечённостью, а также финансовой устойчивостью бизнеса и оценкой маркетинговых активностей.

* **Average Bet Size & Bet Frequency.** Данные метрики подходят для оценки поведения игроков, сегментации пользователей (RFM-анализа), например, для дальнейшего более персонализированного маркетингового предложения.

* **Win/Loss Ratio.** Данная метрика показывает честность и увлекательность игрового опыта, соблюдение такого баланса игры, которые мотивирует пользователя продолжать играть. Также данный показатель хорош для отслеживания технических сбоев или фрода.

**2. На основании имеющихся таблиц необходимо написать SQL-запросы**

In [ ]:
# который посчитает количество уникальных устройств
# и уникальных пользователей по месяцам
# в период с Июля-2023 по Декабрь-2023 (включительно)

'''
    SELECT  toStartOfMonth(event_timestamp) AS month,
            COUNT(DISTINCT player_id) AS players_cnt,
            COUNT(DISTINCT gadget_id) AS gadgets_cnt

    FROM event_log

    WHERE toStartOfMonth(event_timestamp) BETWEEN toDate('2023-07-01') AND toLastDayOfMonth(toDate('2023-12-01'))

    GROUP BY 1
'''

In [ ]:
# который выведет ТОП-5 стран
# по количеству зарегистрированных пользователей в 2023 году

'''
    SELECT  country_code,
            COUNT(DISTINCT player_id) AS players_cnt

    FROM user_info

    WHERE toISOYear(record_created) = 2023

    GROUP BY 1

    ORDER BY 2 DESC

    LIMIT 5
'''

In [ ]:
# который выведет ТОП-5 пользователей
# по общей сумме платежей, и по каждой стране за 2023 год

'''
    WITH raw AS (
                 SELECT p.player_id,
                        u.country_code,
                        SUM(p.amount_RUB) AS payments_sum

                FROM user_info AS u

                INNER JOIN payment_records AS p
                ON u.player_id = p.player_id
                AND toISOYear(p.payment_completed) = 2023
                AND p.payment_successful = 1

                GROUP BY 1, 2
                )

    SELECT  player_id,
            country_code,
            payments_sum,
            place

    FROM (
          SELECT  player_id,
                  country_code,
                  payments_sum,
                  ROW_NUMBER() OVER(PARTITION BY country_code ORDER BY payments_sum DESC) AS place

                  -- использование ROW_NUMBER() обеспечивает вывод именно 5 пользователей на страну;
                  -- в зависимости от цели можно использовать также DENSE_RANK() или RANK()

          FROM raw
          ) AS a

    WHERE place <= 5

    ORDER BY 2, 3
'''

In [ ]:
# посчитает среднюю продолжительности сессии на сайте за 2023 год

# вариант, когда неизвестны наименования событий для старта и конца сессии
'''
    SELECT  AVG(dateDiff('minute', session_end, session_start)) AS avg_duration_min

    FROM (
          SELECT  play_session_id,
                  MIN(event_timestamp) AS session_start,
                  MAX(event_timestamp) AS session_end

          FROM event_log

          GROUP BY 1

          -- сессии, которые начались и закончились в 2023 году
          HAVING MIN(toISOYear(event_timestamp)) = 2023
          AND MAX(toISOYear(event_timestamp)) = 2023
          ) AS raw
'''

# вариант, когда известны наименования событий для старта и конца сессии
'''
    SELECT  AVG(dateDiff('minute', session_end, session_start)) AS avg_duration_min

    FROM (
          SELECT  play_session_id,
                  MIN(CASE WHEN activity_name = 'session_start' THEN event_timestamp ELSE NULL END) AS session_start,
                  MAX(CASE WHEN activity_name = 'session_end' THEN event_timestamp ELSE NULL END) AS session_end

          FROM event_log

          WHERE activity_name IN ('session_start', 'session_end')
          AND toISOYear(event_timestamp) = 2023

          GROUP BY 1
          ) AS raw

    WHERE session_start IS NOT NULL
    AND session_end IS NOT NULL
'''

In [ ]:
# который посчитает количество уникальных пользователей,
# посетивших сайт (DAU) по месяцам, которые подтвердили свой email-адрес.
# аналитика нужна за период с Июля-2023 по Декабрь-2023 (включительно)

# FYI: в моём запросе считается MAU, т.к. DAU - количество уникальных пользователей,
# посетивших сайт по дням, а не месяцам. Предположу, что это просто опечатка.
'''
    SELECT  toStartOfMonth(event_timestamp) AS month,
            COUNT(DISTINCT player_id) AS players_cnt

    FROM event_log AS e

    INNER JOIN user_info AS u
    ON e.player_id = u.player_id
    AND toISOYear(email_verified) <> 1970

    WHERE toStartOfMonth(event_timestamp) BETWEEN toDate('2023-07-01') AND toLastDayOfMonth(toDate('2023-12-01'))

    GROUP BY 1
'''

In [ ]:
# который позволит оценить долю (по сумме успешного депозита)
# пользователей с подтвержденной почтой, в Декабре 2023 года

# FYI: перефразирую, как я поняла.
# доля пользователей с подтверждённой почтой
# среди пользователей с успешным внесённым депозитом в декабре 2023
'''
    SELECT  COUNT(player_id) AS total_payed_players_cnt,
            SUM(email_verified_flag) AS verified_payed_players_cnt,
            (SUM(email_verified_flag) / COUNT(player_id)) * 100 AS verified_payed_players_percent

    FROM (
          SELECT  p.player_id,
                  MAX(CASE WHEN toISOYear(u.email_verified) <> 1970 THEN 1 ELSE 0 END) AS email_verified_flag

          FROM payment_records AS p

          LEFT JOIN user_info AS u
          ON p.player_id = u.player_id

          WHERE toStartOfMonth(p.payment_initiated) = toDate('2023-12-01')
          AND p.payment_successful = 1

          GROUP BY 1
          ) AS raw
'''

In [ ]:
# который выведет для каждого пользователя
# среднее значение,
# нижний квартиль,
# медиану,
# верхний квартиль
# по времени между успешными депозитами.

'''
    WITH raw AS (
                SELECT  player_id,
                        dateDiff(
                                'second',
                                payment_initiated,
                                LAG(payment_initiated) OVER(PARTITION BY player_id ORDER BY payment_initiated, transaction_id)
                                ) AS payments_diff

                FROM (
                      SELECT  DISTINCT
                              player_id,
                              transaction_id,
                              payment_initiated

                      FROM payment_records

                      WHERE payment_successful = 1
                      AND toISOYear(payment_completed) <> 1970
                      ) AS r
                )

    SELECT  DISTINCT
            player_id,
            AVG(payments_diff) OVER(PARTITION BY player_id) AS avg_diff_sec,
            quantileExact(0.25)(payments_diff) OVER (PARTITION BY player_id) AS lower_quartile_diff_sec,
            quantileExact(0.5)(payments_diff) OVER (PARTITION BY player_id) AS median_diff_sec,
            quantileExact(0.75)(payments_diff) OVER (PARTITION BY player_id) AS upper_quartile_diff_sec

    FROM raw
'''

**3.1. Манипуляции данными в Python**

Есть таблица с действиями пользователей A/B-теста:

$events$ - посещение пользователем сайта
- $user\_id$ - id юзера,
- $user\_group$ - группа A/B-теста,
- $time$ - время совершения действия,

Пользовательская сессия определяется по следующим правилам:

Новая сессия начинается после 30 минут бездействия.

<b>Постройте таблицу с сессиями юзеров в формате<b>:
<br>
- $user\_id$ - id юзера
- $ab\_group$ - группа A/B-теста,
- $session\_start$ - время старта сессии,
- $session\_end$ - время окончания сессии,
- $session\_date$ - дата сессии.

In [ ]:
# загрузка библиотек

import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
# читает данные и записывает в переменную
df = pd.read_parquet('task_3_events.parquet')

In [ ]:
# создание функции для первичного обзора данных

def first_check(data):

    print('Первые 10 строк таблицы \n')
    display(data.head(10))

    print('-' * 50)
    print('Последние 10 строк таблицы \n')
    display(data.tail(10))

    print('-' * 50)
    print('Информация о таблице \n')
    print(data.info())

    print('-' * 50)
    print('Количество дубликатов:', data.duplicated().sum())

In [ ]:
# первичный обзор данных
first_check(df)

Первые 10 строк таблицы 



,user_id,user_group,time
0,test_user_68722185,0,2023-12-01 00:00:02.465889
1,test_user_50279722,1,2023-12-01 00:00:02.490361
2,test_user_68722185,0,2023-12-01 00:00:03.803897
3,test_user_68721973,0,2023-12-01 00:00:03.821829
4,test_user_68721973,0,2023-12-01 00:00:03.854164
5,test_user_58080739,0,2023-12-01 00:00:05.630186
6,test_user_68722344,0,2023-12-01 00:00:06.246549
7,test_user_68167487,0,2023-12-01 00:00:07.502947
8,test_user_68722344,0,2023-12-01 00:00:07.852709
9,test_user_68478703,1,2023-12-01 00:00:08.405711


--------------------------------------------------
Последние 10 строк таблицы 



,user_id,user_group,time
835347,test_user_70335346,0,2023-12-12 09:31:21.011998
835348,test_user_18844468,1,2023-12-12 09:31:21.106580
835349,test_user_70339215,1,2023-12-12 09:31:21.199939
835350,test_user_70335346,0,2023-12-12 09:31:21.282926
835351,test_user_47517627,1,2023-12-12 09:31:21.294108
835352,test_user_70339321,1,2023-12-12 09:31:21.370902
835353,test_user_70339109,1,2023-12-12 09:31:21.472440
835354,test_user_47517627,1,2023-12-12 09:31:21.474779
835355,test_user_47517627,1,2023-12-12 09:31:21.486431
835356,test_user_47517627,1,2023-12-12 09:31:21.501760


--------------------------------------------------
Информация о таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835357 entries, 0 to 835356
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     835357 non-null  object        
 1   user_group  835357 non-null  int64         
 2   time        835357 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 19.1+ MB
None
--------------------------------------------------
Количество дубликатов: 22


In [ ]:
# удаляет дубликаты и сортирует данные по id пользователя и времени события
df = df.drop_duplicates().sort_values(by=['user_id', 'time'])

In [ ]:
# добавляет поле со временем следующего события
df['next_event_time'] = df.groupby('user_id')['time'].shift(-1)

In [ ]:
# определяет промежуток времени между событиями
df['time_diff'] = df['next_event_time'] - df['time']

In [ ]:
# считает количество минут между событиями
df['time_diff_int'] = (df['time_diff']).dt.total_seconds() / 60

In [ ]:
# заменяет пропуски на 30 минут (нет следующего события)
df['time_diff_int'] = df['time_diff_int'].fillna(30)

In [ ]:
# определяет строку со временем окончания сессии, выводит время
df['end'] = pd.to_datetime(
    np.where(df['time_diff_int'] >= 30, df['time'], pd.NaT)
)

In [ ]:
# заполняет время окончания по первому заполненному значению снизу вверх
df['end'] = df.sort_values(by=['user_id', 'time'], ascending=True)['end'].bfill()

In [ ]:
# определяет старт каждой сессии через группировку по времени окончания
df = df.groupby(['user_id', 'user_group', 'end']).agg({'time': 'min'}).reset_index()

In [ ]:
# переименовывает поля
df.columns = ['user_id', 'ab_group', 'session_end', 'session_start']

In [ ]:
# создаёт колонку с датой старта сессии
df['session_date'] = df['session_start'].dt.date

In [ ]:
# повторный обзор данных
first_check(df)

Первые 10 строк таблицы 



,user_id,ab_group,session_end,session_start,session_date
0,test_user_10021823,1,2023-12-07 01:10:11.804765,2023-12-07 01:08:51.008309,2023-12-07
1,test_user_10030144,0,2023-12-02 20:38:14.301560,2023-12-02 20:37:34.861127,2023-12-02
2,test_user_1003396,1,2023-12-01 00:01:41.376842,2023-12-01 00:00:48.238407,2023-12-01
3,test_user_1003396,1,2023-12-02 08:35:03.351138,2023-12-02 08:21:13.134305,2023-12-02
4,test_user_1003396,1,2023-12-02 21:34:53.437896,2023-12-02 21:12:09.512252,2023-12-02
5,test_user_1003396,1,2023-12-03 14:46:22.207067,2023-12-03 14:33:21.477400,2023-12-03
6,test_user_1003396,1,2023-12-03 15:43:22.968943,2023-12-03 15:40:25.552902,2023-12-03
7,test_user_1003396,1,2023-12-05 14:40:58.538497,2023-12-05 14:40:07.453175,2023-12-05
8,test_user_1003396,1,2023-12-06 16:27:58.916228,2023-12-06 15:08:48.958667,2023-12-06
9,test_user_1003396,1,2023-12-10 20:51:41.616399,2023-12-10 20:48:47.261972,2023-12-10


--------------------------------------------------
Последние 10 строк таблицы 



,user_id,ab_group,session_end,session_start,session_date
25034,test_user_9921706,0,2023-12-07 17:15:48.268616,2023-12-07 16:20:36.048106,2023-12-07
25035,test_user_9935486,0,2023-12-14 13:13:00.962182,2023-12-14 13:11:43.288526,2023-12-14
25036,test_user_9963470,0,2023-12-03 16:37:53.674279,2023-12-03 16:36:05.322192,2023-12-03
25037,test_user_9964265,0,2023-12-10 06:20:41.221201,2023-12-10 06:20:35.149162,2023-12-10
25038,test_user_9964265,0,2023-12-12 09:07:56.116802,2023-12-12 09:07:28.971166,2023-12-12
25039,test_user_997142,0,2023-12-13 19:35:07.340102,2023-12-13 19:11:15.701582,2023-12-13
25040,test_user_997142,0,2023-12-13 21:04:34.052487,2023-12-13 20:44:51.855149,2023-12-13
25041,test_user_999580,0,2023-12-03 15:06:14.859587,2023-12-03 14:59:54.018927,2023-12-03
25042,test_user_999580,0,2023-12-06 15:42:07.074273,2023-12-06 15:41:35.592334,2023-12-06
25043,test_user_9998026,1,2023-12-06 15:36:20.240469,2023-12-06 15:36:14.988054,2023-12-06


--------------------------------------------------
Информация о таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25044 entries, 0 to 25043
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        25044 non-null  object        
 1   ab_group       25044 non-null  int64         
 2   session_end    25044 non-null  datetime64[ns]
 3   session_start  25044 non-null  datetime64[ns]
 4   session_date   25044 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 978.4+ KB
None
--------------------------------------------------
Количество дубликатов: 0


**3.2. Оценка АБ-теста**

Был проведен A/B-тест.
<br>
В качестве данных используйте таблицу, построенную в предыдущем задании. Первая сессия юзера считается моментом попадания в A/B-тест.
<br><br>
Ключевая метрика эксперимента - конверсия во вторую сессию.
<br>
Сделайте вывод о том, какая группа выиграла в A/B-тесте. Ответ обоснуйте.
<br>

In [ ]:
# создаёт датафрейм с количеством сессий на пользователя
ab_df = (
    df.groupby(['user_id', 'ab_group']).agg({'session_start': 'count'})
    .reset_index()
    .rename(columns={'session_start': 'session_cnt'})
)

In [ ]:
# устанавливает флаг для пользователей с двумя и более сессиями
ab_df['conversion_flag'] = np.where(ab_df['session_cnt'] >= 2, 1, 0)

In [ ]:
# проверка на пересечение пользователей в группах
ab_df.groupby('user_id').agg({'ab_group': 'nunique'}).reset_index().query('ab_group > 1')

,user_id,ab_group


In [ ]:
# проверка на отсутствие дубликатов пользователей
ab_df['user_id'].nunique() == ab_df['user_id'].count()

True

In [ ]:
# посмотрим, в какой период проводился эксперимент

print('min date:', df['session_date'].min())
print('max date:', df['session_date'].max())

min date: 2023-12-01
max date: 2023-12-14


In [ ]:
# поскольку в условии не дана информация о том, какая была задана длительность эксперимента
# и минимальное количество пользователей на группу, будем считать, что данные показатели
# соответствую заданным перед началом теста.

# также предположим, что группа 0 - это контрольная группа,
# а группа 1 - тестовая.

In [ ]:
# проведём тест на Sample Ratio Mismatch (SRM)
# выведим основные показатели о количестве пользователей

print(
    'Общее количество пользователей в двух группах:',
    ab_df['user_id'].count()
    )

print(
    'Количество пользователей в одной группе, если бы распределение было 50/50:',
    ab_df['user_id'].count() / 2
    )

print(
    'Количество пользователей в группе 0 (контрольной):',
    ab_df.loc[ab_df['ab_group'] == 0]['user_id'].nunique()
    )

print(
    'Количество пользователей в группе 1 (тестовой):',
    ab_df.loc[ab_df['ab_group'] == 1]['user_id'].nunique()
    )

Общее количество пользователей в двух группах: 14828
Количество пользователей в одной группе, если бы распределение было 50/50: 7414.0
Количество пользователей в группе 0 (контрольной): 7429
Количество пользователей в группе 1 (тестовой): 7399


In [ ]:
# H0: Наблюдаемое отношение количества пользователей в тестовой группе
# к количеству пользователей в контрольной группе не отличается
# от ожидаемого отношения 50/50.

# H1: Наблюдаемое отношение количества пользователей в тестовой группе
# к количеству пользователей в контрольной группе отличается
# от ожидаемого отношения 50/50.

In [ ]:
# ожидаемое количество выборок для двух групп
expected_counts = [
    ab_df['user_id'].nunique() / 2,
    ab_df['user_id'].nunique() / 2
    ]

# фактическое количество выборок для двух групп
actual_counts = [
    ab_df.loc[ab_df['ab_group'] == 0]['user_id'].nunique(),
    ab_df.loc[ab_df['ab_group'] == 1]['user_id'].nunique()
    ]

# Вычисление статистики хи-квадрат и p-значения
chi2, p_value = stats.chisquare(f_obs=actual_counts, f_exp=expected_counts)

print(f'Chi-squared Statistic: {chi2}')
print(f'P-value: {p_value}')

# Проверка наличия Sample Ratio Mismatch
if p_value < 0.05:
    print('Есть Sample Ratio Mismatch (SRM). Не принимаем Н0, разбиение невалидно')
else:
    print('Нет Sample Ratio Mismatch (SRM). Принимаем Н0, разбиение валидно.')

Chi-squared Statistic: 0.060695980577286215
P-value: 0.8053992773330002
Нет Sample Ratio Mismatch (SRM). Принимаем Н0, разбиение валидно.


In [ ]:
# проведём z-тест для оценки результатов эксперимента
# выведим основные показатели о количестве пользователей с конверсией

print(
    'Общее количество пользователей в группе 0 (контрольной):',
    ab_df.loc[ab_df['ab_group'] == 0]['user_id'].nunique()
    )

print(
    'Количество пользователей в группе 0 (контрольной) с двумя и более сессиями:',
    ab_df.loc[ab_df['ab_group'] == 0]['conversion_flag'].sum()
    )

print(
    'Конверсия во вторую сессию в группе 0 (контрольной):',
    (
        ab_df.loc[ab_df['ab_group'] == 0]['conversion_flag'].sum() /
        ab_df.loc[ab_df['ab_group'] == 0]['user_id'].nunique()
    )
)

print()

print(
    'Общее количество пользователей в группе 1 (тестовой):',
    ab_df.loc[ab_df['ab_group'] == 1]['user_id'].nunique()
    )

print(
    'Количество пользователей в группе 1 (тестовой) с двумя и более сессиями:',
    ab_df.loc[ab_df['ab_group'] == 1]['conversion_flag'].sum()
    )

print(
    'Конверсия во вторую сессию в группе 1 (тестовой):',
    (
        ab_df.loc[ab_df['ab_group'] == 1]['conversion_flag'].sum() /
        ab_df.loc[ab_df['ab_group'] == 1]['user_id'].nunique()
    )
)

Общее количество пользователей в группе 0 (контрольной): 7429
Количество пользователей в группе 0 (контрольной) с двумя и более сессиями: 1833
Конверсия во вторую сессию в группе 0 (контрольной): 0.24673576524431282

Общее количество пользователей в группе 1 (тестовой): 7399
Количество пользователей в группе 1 (тестовой) с двумя и более сессиями: 1850
Конверсия во вторую сессию в группе 1 (тестовой): 0.250033788349777


In [ ]:
# H0: конверсии во вторую сессию не отличаются в контрольной и тестовой группах.

# H1: конверсии во вторую сессию отличаются в контрольной и тестовой группах.

In [ ]:
# Контрольная группа: количество конверсий и общий размер выборки
conversions_A = ab_df.loc[ab_df['ab_group'] == 0]['conversion_flag'].sum()
total_A = ab_df.loc[ab_df['ab_group'] == 0]['user_id'].nunique()

# Тестовая группа: количество конверсий и общий размер выборки
conversions_B = ab_df.loc[ab_df['ab_group'] == 1]['conversion_flag'].sum()
total_B = ab_df.loc[ab_df['ab_group'] == 1]['user_id'].nunique()

# Конверсии в процентах
p_A = conversions_A / total_A
p_B = conversions_B / total_B

# Объединенная пропорция
p_pool = (conversions_A + conversions_B) / (total_A + total_B)

# Стандартная ошибка
se = np.sqrt(p_pool * (1 - p_pool) * (1/total_A + 1/total_B))

# Z-статистика
z = (p_A - p_B) / se

# P-значение для двустороннего теста
p_value = 2 * (1 - stats.norm.cdf(abs(z)))

print(f'Z-статистика: {z}')
print(f'P-значение: {p_value}')

# Проверка значимости результата
if p_value < 0.05:
    print('Не принимаем Н0: есть значимая разница в конверсии между группами. Эксперимент можно считать удачным.')
else:
    print('Принимаем Н0: значимой разницы в конверсии между группами нет. Эксперимент можно считать неудачным.')

Z-статистика: -0.4647354263445335
P-значение: 0.6421209354298445
Принимаем Н0: значимой разницы в конверсии между группами нет. Эксперимент можно считать неудачным.
